In [4]:
#import statements for the models to work
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import datasets, cluster

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

## Installing weights and biases and logging in 

In [5]:
%%capture
!pip install wandb
import wandb
#installing wandb into the environment

In [6]:
wandb.login() #login in another cell to get credentials from the browser login 

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## MLP implementation for cifar-10

In [7]:
#cifar specific import statments
from keras.datasets import cifar10
from sklearn import metrics
from sklearn.model_selection import cross_val_score    
#printing start time to check when training concludes
import time
start_time = time.time()
print(start_time)


# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# X_train is 50000 rows of 3x32x32 values --> reshaped in 50000 x 3072
RESHAPED = 3072

X_train = X_train.reshape(50000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
#flattened into a vector
y_train = y_train.flatten()
y_test = y_test.flatten()

# normalize the datasets
X_train /= 255.
X_test /= 255.
#printing metadata
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_train.shape)
print(y_train.shape)

#training

mlp2 = MLPClassifier(hidden_layer_sizes=(300,450,300), max_iter=10, alpha=1e-4, 
                    solver='sgd', verbose=10, random_state=420, learning_rate_init=.01)
mlp2.fit(X_train, y_train)
# make predictions
expected_y  = y_test
predicted_y = mlp2.predict(X_test)

50000 train samples
10000 test samples
(50000, 3072)
(50000,)
Iteration 1, loss = 1.84552940
Iteration 2, loss = 1.62388828
Iteration 3, loss = 1.52846408
Iteration 4, loss = 1.46998909
Iteration 5, loss = 1.41750600
Iteration 6, loss = 1.37560290
Iteration 7, loss = 1.33670737
Iteration 8, loss = 1.30260850
Iteration 9, loss = 1.27138013
Iteration 10, loss = 1.24094100


In [8]:
print(mlp2.score(X_train,y_train))
print(mlp2.score(X_test,y_test))
#checking score

0.5827
0.5217


#Uploading to weights and biases after settting experiment metadata

In [9]:
run = wandb.init(project='cifar10-class-1', name="class-1")
#initializing the object to add in wandb experiment

wandb: Currently logged in as: vikasmehta. Use `wandb login --relogin` to force relogin


##Plotting the Curves in logged in account

In [10]:
y_pred = mlp2.predict(X_test)
y_probas = mlp2.predict_proba(X_test)
wandb.sklearn.plot_classifier(mlp2, 
                              X_train, X_test, 
                              y_train, y_test, 
                              y_pred, y_probas, 
                              np.unique(y_train), 
                              is_binary=False, 
                              model_name='MLP-Cifar')

wandb.finish()
#upload to wandb dashboard and generating curves

wandb: 
wandb: Plotting MLP-Cifar.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: WARNING This function only supports binary classification at the moment and therefore expects labels to be binary. Skipping calibration curve.
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.
